In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
def parse(games, prod):
    i = 0
    l = len(games)
    result = pd.DataFrame()
    if len(prod) > 0:
        pl = ['', '-'+prod.replace(' ','-')]
    else: pl = ['']
    for p in pl:
        for game in games:
            g = game.replace(' ','-')
            url = f"https://slotcatalog.com/en/slots/{g}{p}"
            response = requests.get(url)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                slot_attr_review = soup.find('div', class_='slotAttrReview')

                if slot_attr_review:
                    hcon_page = slot_attr_review.find('div', class_='HConPage')
                    if hcon_page:
                        hcon_page.decompose()

                    data = {}
                    game_title = soup.find('h3', id='attributes')
                    if game_title:
                        data['Game Title'] = game_title.get_text(separator=" ", strip=True).replace(' Attributes', '')

                    for row in slot_attr_review.find_all('tr'):
                        header = row.find('th', class_='propLeft')
                        value_cell = row.find('td', class_='propRight')
                        if header and value_cell:
                            key = header.text.strip().rstrip(':')
                            value = value_cell.get_text(separator=" ", strip=True)
                            data[key] = value
                        elif 'colspan' in row.find('td').attrs:
                            key, value = row.find('td').text.split(':', 1)
                            key = key.strip()
                            value = value.strip()
                            if value.startswith('N/A'):
                                value = 'N/A'
                            elif row.find('td').find('a'):
                                value = ', '.join(a.get_text(strip=True) for a in row.find('td').find_all('a'))
                            data[key] = value
                else:
                    print("Элемент не найден")

                tt = pd.DataFrame([data])
                result = pd.concat([result,tt],axis=0)

            i+=1
            sys.stdout.write(f'\rЭто итерация номер {i} / {l*len(pl)}')
            time.sleep(1)
            sys.stdout.flush()
    return result

In [ ]:
games_to_parse = [list_of_games_names]
pg_soft = parse(games_to_parse,'PG Soft')